In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/plants/plants.data

--2021-10-04 14:47:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/plants/plants.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1690800 (1.6M) [application/x-httpd-php]
Saving to: ‘plants.data.1’

plants.data.1       100%[===================>]   1.61M  5.51MB/s    in 0.3s    

2021-10-04 14:47:26 (5.51 MB/s) - ‘plants.data.1’ saved [1690800/1690800]



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian, Bernoulli


state_names = set()
for line in open('plants.data', encoding='ISO-8859-1'):
    name, *states = line.rstrip().split(",")
    state_names.update(states)

features = list(sorted(list(state_names)))
rows = []
for line in open('plants.data', encoding='ISO-8859-1'):
    name, *states = line.rstrip().split(",")
    row = np.zeros(len(features))
    for state in states:
        row[features.index(state)] = 1
    rows.append(np.array(row))

D = np.array(rows)
D = D[D.sum(axis = 1)  >= 2]
ptypes = [Bernoulli] * len(features)
categorical = [True]*len(features)

train, test = train_test_split(D, random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

17411 in train, 5804 in test


In [2]:
D.shape

(23215, 70)

In [3]:
net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe=True,
                       min_instances_slice = len(train) / 100)

/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iteration

/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


In [5]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product
names = features
def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    
    if ((sign, b) == ('==', '1')) or ((sign, b) == ('!=', '0')):
        return ("%s" % (a))
    else:
        return ("~%s" % (a))

print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))
net.n_instances = len(train)
csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.1, max_depth = 2)

rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  342
342, 9.60, 9.61
23, 6.22, 7.09


In [8]:
from spn.algorithms.Inference import log_likelihood

print ("test log-likelihood = %.2f" % np.mean(log_likelihood(net, test)))

test log-likelihood = -14.03
